# 🧪 pipeline.ipynb — CI local en Notebook
Ejecuta localmente el pipeline con Lint, SAST, Auditoría (opcional) y 3 suites de tests (unitarias, seguridad, integración).
Coloca este archivo en la **raíz del proyecto**.


In [ ]:
SKIP_AUDIT = False
TEST_LOG = 'pytest.log'
print('SKIP_AUDIT =', SKIP_AUDIT, '| TEST_LOG =', TEST_LOG)


In [ ]:
import sys, subprocess, pathlib
assert pathlib.Path('requirements.txt').exists(), 'No se encontró requirements.txt'
print('Python:', sys.executable)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'])
print('Deps OK')


In [ ]:
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'ruff', 'check', 'app']); print('Ruff OK')
subprocess.check_call([sys.executable, '-m', 'bandit', '-q', '-r', 'app', '-x', 'tests']); print('Bandit OK')
if not SKIP_AUDIT:
    subprocess.check_call([sys.executable, '-m', 'pip_audit', '--strict']); print('pip-audit OK')
else:
    print('Saltando pip-audit')


In [ ]:
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pytest', '-v', 'tests/test_password_hash.py', 'tests/test_validation.py', '--log-file', 'pytest.log', '--log-level', 'INFO'])
subprocess.check_call([sys.executable, '-m', 'pytest', '-v', 'tests/test_jwt.py', 'tests/test_ratelimit.py', 'tests/test_security_attacks.py', '--log-file', 'pytest.log', '--log-level', 'INFO'])
subprocess.check_call([sys.executable, '-m', 'pytest', '-v', 'tests/test_integration_auth.py', 'tests/test_auth.py', '--log-file', 'pytest.log', '--log-level', 'INFO'])
print('Tests OK')


In [ ]:
from pathlib import Path
print('coverage_html:', Path('coverage_html/index.html').resolve())
logf = Path('pytest.log'); print('pytest.log:', logf.resolve())
if logf.exists():
    print('\n--- Ultimas 60 lineas ---\n'); print('\n'.join(logf.read_text(encoding='utf-8', errors='ignore').splitlines()[-60:]))
else:
    print('pytest.log no encontrado')
